# Face-Recognition
Este notebook tem como objetivo realizar a inferência utilizando pesos de modelos treinados dentre os presentes no framework face-recognition. Aqui é possível: Calcular a similaridade entre duas imagens e extrair embeddings de uma imagem ou em batch.

## Importações e Inicialização

In [1]:
import sys
import torch
import numpy as np
import os
import json
from pathlib import Path

sys.path.append('../../src/models/face-recognition')
from inference import get_network, load_model, compare_faces, extract_features, extract_batch_embeddings
from models import (
    sphere20,
    sphere36,
    sphere64,
    MobileNetV1,
    MobileNetV2,
    mobilenet_v3_small,
    mobilenet_v3_large
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


/home/ubuntu/miniconda3/envs/face_rec_py310/lib/python3.10/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Selecionar Modelo

In [2]:
# Modelos disponíveis
available_models = {
    "sphere20": sphere20,
    "sphere36": sphere36,
    "sphere64": sphere64,
    "mobilenetv1": MobileNetV1,
    "mobilenetv2": MobileNetV2,
    "mobilenetv3_small": mobilenet_v3_small,
    "mobilenetv3_large": mobilenet_v3_large
}

print("Available models:")
for i, model in enumerate(available_models, 1):
    print(f"{i}. {model}")

# Selecionar modelo
model_name = "mobilenetv3_large" # Escolha o modelo desejado aqui
print(f"\nSelected model: {model_name}")

Available models:
1. sphere20
2. sphere36
3. sphere64
4. mobilenetv1
5. mobilenetv2
6. mobilenetv3_small
7. mobilenetv3_large

Selected model: mobilenetv3_large


## Carregar o Checkpoint

In [3]:
# Configurar caminho do modelo
checkpoint_name = "mobilenetv3_large_5"
model_path = f"../../src/models/face-recognition/weights/{checkpoint_name}.ckpt"

# Carregar modelo usando a função corrigida do inference.py
model = load_model(model_name, model_path, device)

print(f"Model loaded successfully from: {model_path}")
print(f"Selected model: {model_name}")

Model loaded successfully from: ../../src/models/face-recognition/weights/mobilenetv3_large_5.ckpt
Selected model: mobilenetv3_large


## Comparar duas imagens

In [4]:
# Configurações de comparação
img1_path = "../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0001.jpg"  # Substitua pelo caminho da sua imagem
img2_path = "../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0002.jpg"
threshold = 0.35  # Ajuste o limiar conforme necessário

# Comparar as duas imagens
similarity, is_same = compare_faces(model, device, img1_path, img2_path, threshold)

print(f"Image 1: {img1_path}")
print(f"Image 2: {img2_path}")
print(f"Similarity Score: {similarity:.4f}")
print(f"Same Person: {is_same}")
print(f"Threshold: {threshold}")

Image 1: ../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0001.jpg
Image 2: ../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0002.jpg
Similarity Score: 0.6783
Same Person: True
Threshold: 0.35


## Extrair embedding de uma única imagem

In [5]:
# Configurações para extração de embedding
image_path = "../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0002.jpg"  # Mude para o caminho da sua imagem
output_file = "../../assets/single_embedding.json"  # Caminho do output

# Extrair embedding
embedding = extract_features(model, device, image_path)

# Preparar dados para salvamento (mesmo formato do batch)
image_name = Path(image_path).name
embeddings_dict = {
    image_name: embedding.tolist()
}

# Criar diretório se não existir e salvar
os.makedirs(os.path.dirname(output_file), exist_ok=True)
with open(output_file, 'w') as f:
    json.dump(embeddings_dict, f, indent=2)

print(f"Image: {image_path}")
print(f"Embedding shape: {embedding.shape}")
print(f"Embedding (first 10 values): {embedding[:10]}")
print(f"Embedding saved to: {output_file}")

Image: ../../data/raw/lfw/Abdullatif_Sener/Abdullatif_Sener_0002.jpg
Embedding shape: (512,)
Embedding (first 10 values): [-0.6168843  1.9606509 -2.2597706 -1.3064387 -0.894148  -1.5073667
  1.4386004 -0.7539237 -1.8357918  1.2179649]
Embedding saved to: ../../assets/single_embedding.json


## Extrair embeddings em batch

In [6]:
# Configurações para extração em batch
image_folder = "../../data/raw/lfw/Abdullatif_Sener"  # Caminho para a pasta com imagens
output_folder = "../../assets/batch_embeddings"  # Pasta para salvar embeddings individuais

# Criar pasta de output se não existir
os.makedirs(output_folder, exist_ok=True)

# Encontrar todas as imagens
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tiff')
image_files = []

for ext in image_extensions:
    image_files.extend(Path(image_folder).glob(f"*{ext}"))
    image_files.extend(Path(image_folder).glob(f"*{ext.upper()}"))

print(f"Found {len(image_files)} images to process...")

# Processar cada imagem e salvar individualmente
processed_count = 0
for img_path in image_files:
    try:
        # Extrair embedding
        embedding = extract_features(model, device, str(img_path))
        
        # Criar nome do arquivo JSON (sem extensão da imagem)
        image_name_no_ext = img_path.stem  # Nome sem extensão
        json_filename = f"{image_name_no_ext}.json"
        json_path = os.path.join(output_folder, json_filename)
        
        # Criar dicionário com embedding
        embedding_dict = {
            img_path.name: embedding.tolist()
        }
        
        # Salvar arquivo JSON individual
        with open(json_path, 'w') as f:
            json.dump(embedding_dict, f, indent=2)
        
        processed_count += 1
        print(f"✓ Processed and saved: {img_path.name} -> {json_filename}")
        
    except Exception as e:
        print(f"✗ Error processing {img_path.name}: {e}")

print(f"\nExtraction complete!")
print(f"Processed: {processed_count} images")
print(f"Embeddings saved to individual JSON files in: {output_folder}")

Found 2 images to process...
✓ Processed and saved: Abdullatif_Sener_0001.jpg -> Abdullatif_Sener_0001.json
✓ Processed and saved: Abdullatif_Sener_0002.jpg -> Abdullatif_Sener_0002.json

Extraction complete!
Processed: 2 images
Embeddings saved to individual JSON files in: ../../assets/batch_embeddings
